In [1]:
import boto3
s3 = boto3.resource('s3', 'us-east-1')
bucket = s3.Bucket('w210.data')


In [31]:
for obj in bucket.objects.all():
    if obj.key == 'feature/SD_18_26728.pickle':
        test = obj
        obj.Acl().put(ACL='public-read')

In [29]:
# abc=test.Object()
?abc.Acl

In [12]:
for obj in bucket.objects.filter():
    if obj.key == 'online/runtime_data_OR.pickle':
        obj.delete()
        print 'delete done!'

delete done!


In [53]:
for o in bucket.objects.all():
    if o.key.startswith('feature/UT'):
        print o.key
        break
else:
    print 'no found'

feature/UT_74_19243.pickle


In [58]:
bucket.upload_file('UT_74_19243.pickle', 'feature/UT_74_19243.pickle')

In [48]:
bucket.download_file('online/runtime_data_OR.pickle', 'down_test2')

In [55]:
import pickle
with open('UT_74_19243.pickle') as f:
    feature, plans = pickle.load(f)

In [56]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [60]:
import pickle
with open('training/UT_2.pickle') as f:
    plans, rank = pickle.load(f)

In [2]:
from pymongo import MongoClient
local = False
if local:
    client = MongoClient('fc8iasm01', 27017)
    plan_col = client.aca.plan
    drug_col = client.aca.drug
else:
    client = MongoClient('ec2-52-53-230-141.us-west-1.compute.amazonaws.com', 27017)
    plan_col = client.plans.plans
    drug_col = client.formularies.drugs
    prov_col = client.providers.providers
    faci_col = client.providers.facilities

all_plan = drug_col.distinct('plans.plan_id')
all_drug = drug_col.distinct('rxnorm_id')

print 'using %s Mongo, total drug: %d, total plan: %d, total provider: %d' %(
     'local' if local else 'aws', len(all_drug), len(all_plan), prov_col.count())

using aws Mongo, total drug: 46206, total plan: 6035, total provider: 8799098


In [3]:
import numpy as np
state_ids = np.unique([i[5:7] for i in all_plan])
print 'find plan from %d states: %s' %(len(state_ids), ', '.join(state_ids))

find plan from 43 states: AK, AL, AR, AZ, CO, DE, FL, GA, HI, IA, IL, IN, KS, KY, LA, MA, ME, MI, MN, MO, MS, MT, NC, ND, NE, NH, NJ, NM, NV, OH, OK, OR, PA, SC, SD, TN, TX, UT, VA, WA, WI, WV, WY


In [17]:
state_plan = [i for i in all_plan if 'SD' in i]

In [10]:
a=['Glaucoma', 'diabetes Obesophobia', 'Obesophobia diabetes Alzheimer', 'Glaucoma diabetes Alzheimer Hypertension', 'Glaucoma Alzheimer diabetes Obesophobia Hypertension']
np.unique(' '.join(a).split())

array(['Alzheimer', 'Glaucoma', 'Hypertension', 'Obesophobia', 'diabetes'], 
      dtype='|S12')

In [5]:
import pickle
with open('plan_ids.pickle', 'w') as f:
    pickle.dump([all_plan], f)

In [11]:
sim_health = np.array(['Alzheimer', 'Glaucoma', 'Hypertension', 'Obesophobia', 'diabetes'])

In [31]:
np.random.randint(low=1,high=5)

3

In [47]:
def simulate_clicks(n_query = 8, n_click_max = 8):
    '''
    '''

    with open('plan_ids.pickle') as f:
        all_plan = pickle.load(f)

    # sim health
    sim_health = np.array(['Alzheimer', 'Glaucoma', 'Hypertension', 'Obesophobia', 'diabetes'])

    # get states
    sim_data = []
    state_ids = np.unique([i[5:7] for i in all_plan[0]])
    for state in state_ids:
        state_plan = np.array([i for i in all_plan[0] if state in i])
        n_plan = len(state_plan)
#         print 'state %s has %d plans' %(state, n_plan)
        # simulate query        
        for i in range(n_query):
            health = ' '.join(sim_health[np.random.permutation(sim_health.size)[0:i+1]])
            sim_rank = state_plan[np.random.permutation(n_plan)]
            sim_click = state_plan[np.random.permutation(n_plan)][0:np.random.randint(low=1, high=min(n_plan,n_click_max))]
            sim_data.append((state, health, sim_rank, sim_click))

    rtn_type = [('state','S2'), ('query','S512'), ('ranks',list), ('clicks',list)]
    return np.array(sim_data, dtype=rtn_type)

In [56]:
# sdata = simulate_clicks()
sdata[27]

('AZ', 'diabetes Hypertension Glaucoma Obesophobia', [u'53901AZ1320003', u'53901AZ1300003', u'70239AZ0010054', u'53901AZ1150003', u'97667AZ0110005', u'51485AZ0160031', u'91450AZ0160005', u'53901AZ1090003', u'98971AZ0010066', u'53901AZ1090004', u'65441AZ0080002', u'53901AZ0790007', u'70239AZ0010063', u'23307AZ0400015', u'98971AZ0020009', u'65441AZ0050003', u'91450AZ0080026', u'53901AZ1300001', u'78611AZ0060002', u'53901AZ1300010', u'53901AZ0810001', u'53901AZ0810004', u'70239AZ0010071', u'53901AZ0800002', u'53901AZ1300002', u'98971AZ0010065', u'91450AZ0090026', u'53901AZ1350001', u'53901AZ0800005', u'78611AZ0060003', u'65441AZ0080001', u'70239AZ0010057', u'53901AZ1080004', u'70239AZ0010056', u'70239AZ0010069', u'65441AZ0100002', u'98971AZ0020002', u'91450AZ0080031', u'53901AZ0790004', u'70239AZ0010076', u'98971AZ0020005', u'70239AZ0010045', u'53901AZ0810003', u'78611AZ0060005', u'98971AZ0020004', u'97667AZ0110004', u'51485AZ0160005', u'53901AZ1080001', u'53901AZ0810002', u'91450AZ008002